In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown

# Download the files from Google Drive using gdown
#train_data_url = "https://drive.google.com/uc?id=1sOPgJu8NM5ftRHfy2iXo_vM7aKllTvRv"
#item_data_url = "https://drive.google.com/uc?id=1SuHwG29IDqG_qzN7rttJiNXrTM9D6L63"

#train_data_path = 'train_data.csv'  # Specify the path to save the training data
#item_data_path = 'item_data.csv'  # Specify the path to save the item data

#gdown.download(train_data_url, train_data_path, quiet=False)
#gdown.download(item_data_url, item_data_path, quiet=False)

# dynamic training data
train_data = pd.read_csv("/content/drive/MyDrive/data/data/train.csv", sep="|")  # Use the downloaded file path
# static items data
item_data = pd.read_csv("/content/drive/MyDrive/data/data/items.csv", sep="|")  # Use the downloaded file path
# perform join on both the datasets using "pid"
data_new = pd.merge(train_data, item_data, how='left', on='pid')
data_new.shape

In [ ]:
data_new.head()

In [ ]:
# Download class_data
#class_data_url = "https://drive.google.com/uc?id=1tizOGpDeKHmNvVPnNPwUuWwGOfAp1V4f"
#class_data_path = "class_data.csv"
#gdown.download(class_data_url, class_data_path, quiet=False)
class_data = pd.read_csv("/content/drive/MyDrive/data/data/class.csv", sep="|")
class_data.set_index('lineID', inplace=True, drop=True)

# Download real_class_data
#real_class_data_url = "https://drive.google.com/uc?id=1dVEn6zuMIvBcUt0hvskaMix8VqBLoVqs"
#real_class_data_path = "real_class_data.csv"
#gdown.download(real_class_data_url, real_class_data_path, quiet=False)
real_class_data = pd.read_csv("/content/drive/MyDrive/data/data/realclass.csv", sep="|")
real_class_data.rename(columns={'revenue': 'actual_revenue'}, inplace=True)
real_class_data.set_index("lineID", inplace=True, drop=True)

# Join and merge the data
test_data = class_data.join(real_class_data, on='lineID', how='inner')
test_data = pd.merge(test_data, item_data, how='left', on='pid')  # Assuming item_data is already loaded
test_data.shape

In [ ]:
test_data.head()

In [ ]:
data_new.info()

In [ ]:
features_null_values=[features for features in data_new.columns if data_new[features].isnull().sum()>1]
for i in features_null_values:
    print(i, np.round(data_new[i].isnull().mean()*100,4), ' % missing values')

In [ ]:
import datetime

def gen_dates(month, year):
    start_date = datetime.date(year, month, 1)
    dates = [start_date]

    for _ in range(91):
        start_date += datetime.timedelta(days=1)
        dates.append(start_date)

    return dates

# Example: Generating dates for a hypothetical month with 92 days in the year 2024
month = 10  # October
year = 2024

date_list = gen_dates(month, year)

# # Print the list of dates
# for date in date_list:
#     print(date.strftime("%Y-%m-%d"))

final_dates = []

for index in np.array( np.unique(data_new['day']), dtype=int):
    for i in range((data_new['day'] == index).sum()):
        final_dates.append( date_list[index-1] )

print(len(final_dates))


In [ ]:
data_new["Date"] = final_dates

In [ ]:
pd.pandas.set_option('display.max_columns',None)

In [ ]:
data_new.head()

## Exploratory Data Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

#### 1. **Revenue Distribution**
**Variable:** Revenue  
**Graph:** Histogram  
**Question:** How is revenue distributed based on each user action?  
**Answer:** The revenue distribution per user action is strongly left-skewed, as seen by the histogram. Most activities yield extremely little revenue, usually between 0 and 1 units.

In [ ]:
fig_1 = px.histogram(data_new, x='revenue', title='Revenue Distribution')
fig_1.update_xaxes(title_text='Revenue', range=[0, 600])  # Adjust the x-axis range as needed
fig_1.update_yaxes(title_text='Frequency', range=[0, 32000])  # Adjust the y-axis range as needed
fig_1.show()

In [ ]:
fig_1_log = px.histogram(x=np.log(data_new['revenue']), title='Revenue Distribution')
fig_1_log.update_xaxes(title_text='Log of Revenue')
fig_1_log.update_yaxes(title_text='Frequency')


#### 2. **Action Type Distribution**
**Variable:** Revenue  
**Graph:** Histogram  
**Question:** What is the distribution of user actions (click, basket, order)?  
**Answer:** The histogram illustrates the substantial left-skewed revenue distribution per user action. The majority of activities generate very little income, typically in the range between 0 and 1.

In [ ]:
action_counts = data_new[['click', 'basket', 'order']].sum()
fig_2 = px.bar(x=action_counts.index, y=action_counts.values, title='Action Type Distribution')
fig_2.update_xaxes(title_text='Action Type')
fig_2.update_yaxes(title_text='Count')

### 3. **Daily Revenue Trends**

**Variables:** Date, Revenue  
**Graph:** Line Plot  
**Question:** What is the time variation in revenue?  
**Answer:** Over time, there are noticeable peaks and dips in the revenue, with a pattern that repeats every seven days.


In [ ]:
revenue_daily = data_new.groupby('day')['revenue'].sum().reset_index()
fig_3 = px.line(revenue_daily, x='day', y='revenue', title='Daily Revenue Trend')
fig_3.update_xaxes(title_text='Day Number')
fig_3.update_yaxes(title_text='Revenue')

### 4. **Competitor's Price vs. Our Price**

**Variables:** Competitor's Price, Our Price  
**Graph:** Scatter Plot  
**Question:** Does our price compare favorably with that of our competitors?  
**Answer:** Over the whole revenue range, there is a positive linear correlation between our pricing and the competitor's price.


In [ ]:
# Create the scatter plot with Plotly
fig = px.scatter(data_new, x='price', y='competitorPrice', color='revenue',
                 title='Analysis 4: Competitor\'s Price vs. Our Price',
                 labels={'price': 'Our Price', 'competitorPrice': 'Competitor\'s Price', 'revenue': 'Revenue'},
                 width=1000, height=600)

# Update layout for a more polished look
fig.update_layout(
    title={'x':0.5, 'xanchor': 'center'},  # Center the title
    xaxis_title='Our Price',
    yaxis_title='Competitor\'s Price',
    template='plotly_white',  # Cleaner background style
    coloraxis_colorbar=dict(title='Revenue')
)

# Optimize for performance
fig.update_traces(marker=dict(size=5))  # Reduce marker size
fig.update_layout(hovermode='closest')  # Optimize hover performance

# Show the plot
fig.show()

### 5. **Click-Through Rate (CTR) Analysis**

**Variables:** Click and Basket  
**Graph:** Bar Chart  
**Question:** How is the CTR(Click Through Rate) variable?  
**Answer:** Out of 2.77 million clicks, 1.17 million result in the addition of a basket, or about 42% of clicks.


In [ ]:
df_temp = pd.DataFrame()
df_temp["revenue"] = data_new["revenue"]
df_temp['CTR'] = (data_new["click"]+data_new["basket"]+data_new["order"])/(data_new["basket"]+data_new["order"])


df_temp.dropna(inplace=True)
temp = df_temp["CTR"]

for index in temp.index:
    if temp[index]==1.0:
        temp[index]='yes'
    else:
        temp[index]='no'

fig = px.histogram(x=temp)
fig.update_layout(bargap=0.2)
fig.update_layout(
    title=" Click-Through Rate (CTR) Analysis",
    xaxis_title="Added to basket after Clicking?",
    yaxis_title="Count")
fig.show()

In [ ]:
df_temp["CTR"] = temp
CTR_revenue = df_temp.groupby('CTR')['revenue'].sum().reset_index()
# Create the bar plot with Plotly
fig = px.bar(CTR_revenue, x='CTR', y='revenue',
             title='Analysis 5: Click-Through Rate (CTR) Analysis',
             labels={'CTR': 'CTR', 'revenue': 'Revenue'},
             width=800, height=500)

# Update layout for a more polished look
fig.update_layout(
    title={'x':0.5, 'xanchor': 'center'},  # Center the title
    xaxis_title='CTR',
    yaxis_title='Revenue',
    template='plotly_white'  # Optional: cleaner background style
)

# Show the plot
fig.show()

### 6. **Basket Conversion Rate**

**Variables:** Basket, Revenue  
**Graph:** Bar Chart  
**Question:** What impact does the basket conversion rate have on sales?  
**Answer:** Of the 1.2 million basket additions, almost 60% (or 0.7 million) result in a purchase.


In [ ]:

# Prepare DataFrame
df_temp = pd.DataFrame()
df_temp["basket"] = data_new['basket']
df_temp['revenue'] = data_new['revenue']
df_temp['order'] = data_new['order']
df_temp["click"] = data_new['click']
df_temp['Basket Conversion Rate'] = (df_temp['basket'] + df_temp['order']) / df_temp['order']

# Drop NaN values
df_temp.dropna(inplace=True)

# Categorize Basket Conversion Rate
df_temp['Basket Conversion Rate'] = df_temp['Basket Conversion Rate'].apply(lambda x: 'Yes' if x == 1.0 else 'No')

# Plot histogram
fig = px.histogram(df_temp, x='Basket Conversion Rate', title="Basket Conversion Rate (BCR) Analysis")
fig.update_layout(bargap=0.2)
fig.update_xaxes(title="Ordered after adding to basket?")
fig.update_yaxes(title="Count")
fig.show()


### 7. **Product Category Analysis**

**Variable:** Product Category  
**Graph:** Bar Chart  
**Question:** Which product categories yield the highest income?
  
**Answer:**  
The product category with the highest revenue generation, product category 50, brings in about 3 lakhs, followed by product category 99 with about 2.75 lakhs and product category 25 with about 2.4 lakhs. Nearly no money is made by the product categories 327, 337, 364, 379, and 391.



In [ ]:

# Group by category and calculate total revenue
category_revenue = data_new.groupby('category')['revenue'].sum().reset_index()

# Create the bar plot with Plotly
fig = px.bar(category_revenue, x='category', y='revenue',
             title='Analysis 7: Product Category Analysis',
             labels={'category': 'Product Category', 'revenue': 'Revenue'},
             width=800, height=500)

# Update layout for better aesthetics
fig.update_layout(
    title={'x': 0.1, 'xanchor': 'center'},  # Center the title
    xaxis_title='Product Category',
    yaxis_title='Revenue',
    template='plotly_white'  # Optional: cleaner background style
)

# Show the plot
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Group by category and calculate total revenue
category_revenue = data_new.groupby('category')['revenue'].sum().reset_index()

# Sort values by revenue in descending order
category_revenue.sort_values(by='revenue', ascending=False, inplace=True)

# Select the top 20 categories
category_revenue = category_revenue.head(20)

# Create the bar plot with Plotly
fig = px.bar(category_revenue, x='category', y='revenue',
             title='Analysis 7: Top 20 Product Categories in Terms of Revenue',
             labels={'category': 'Product Category', 'revenue': 'Revenue'},
             width=800, height=500)

# Update traces to add thickness to the bars
#fig.update_traces(marker=dict(line=dict(width=2)))  # Set the thickness of the bars

# Update layout for better aesthetics
fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},  # Center the title
    xaxis_title='Product Category',
    yaxis_title='Revenue',
    template='plotly_white'  # Optional: cleaner background style
)

# Show the plot
fig.show()


In [ ]:
category_revenue = data_new.groupby('category')['revenue'].sum().reset_index()
category_revenue.sort_values(by='revenue', ascending=False, inplace=True)
category_revenue = category_revenue[-20:]
plt.figure(figsize=(10, 6))
sns.barplot(data=category_revenue, x='category', y='revenue')
plt.title('Analysis 7: Last 20 Product Category in terms of Revenue')
plt.xlabel('Product Category')
plt.ylabel('Revenue')

## 8. **User Behavior by Day of the Week**

**Variables**: Date, Click, Basket, Order  
**Graph**: Heatmap  
**Question**: Does the day of the week affect the actions of users?  
**Sample Answer**:  
On Mondays, the most clicks occur, and on Saturdays, the least. Tuesdays and Sundays had the most clicks and orders among users, after Mondays.


In [ ]:
data_new['Date'] = pd.to_datetime(final_dates)
data_new['Day of Week'] = data_new['Date'].dt.day_name()
action_by_day = data_new[['Day of Week', 'click', 'basket', 'order']].groupby('Day of Week').sum().reset_index()

In [ ]:
plt.figure(figsize=(12, 6))
# Pivot the data for the heatmap
heatmap_data = action_by_day.pivot(index='Day of Week', columns='click', values='order')
sns.heatmap(data=heatmap_data, cmap="YlGnBu", annot=True)
plt.title('Analysis 8: User Behavior by Day of the Week')
plt.xlabel('Action Type: Click')
plt.ylabel('Day of Week')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
# Pivot the data for the heatmap
heatmap_data = action_by_day.pivot(index='Day of Week', columns='order', values='order')
sns.heatmap(data=heatmap_data, cmap="YlGnBu", annot=True)
plt.title('Analysis 8: User Behavior by Day of the Week')
plt.xlabel('Action Type: Order')
plt.ylabel('Day of Week')
plt.show()

In [ ]:
## above graphs are different from that in kaggle notebook

## 9. **Price Elasticity Analysis**

**Variables**: Our Price, Revenue  
**Graph**: Scatter Plot  
**Question**: What effect does raising our price have on sales?

**Sample Answer**:  
Revenue doesn't change in the lower price range (0 to 20). Revenue rises in line with price increases in the mid-range (20 to 200).


In [ ]:
plt.figure(figsize=(10, 6))
df_temp = data_new[data_new['order']==1]
sns.scatterplot(data=df_temp, x='price', y='revenue')
plt.title('Analysis 9: Price Elasticity Analysis - Overall')
plt.xlabel('Our Price')
plt.ylabel('Revenue')


In [ ]:
plt.figure(figsize=(10, 6))
df_temp = data_new[data_new['order']==1]
df_temp = df_temp[df_temp['price']<20]
sns.scatterplot(data=df_temp, x='price', y='revenue')
plt.title('Analysis 9: Price Elasticity Analysis - Low Price Range')
plt.xlabel('Our Price')
plt.ylabel('Revenue')

In [ ]:
plt.figure(figsize=(10, 6))
df_temp = data_new[data_new['order']==1]
df_temp = df_temp[df_temp['price']>20]
df_temp = df_temp[df_temp['price']<200]
sns.scatterplot(data=df_temp, x='price', y='revenue')
plt.title('Analysis 9: Price Elasticity Analysis - Mid Price Range')
plt.xlabel('Our Price')
plt.ylabel('Revenue')

## 10. **User Activity Over Time**

**Variables**: Date, Click, Basket, Order  
**Graph**: Line Plot  
**Question**: What changes have you seen in user activity over time?
  
**Sample Answer**:  
Over time, the general pattern for orders, basket additions, and clicks did not change. User behavior is cyclical, occurring once a week. The middle of the period saw a 15-day decline in user activity. Over time, the percentage of clicks that result in an order or basket addition has been trending in the same direction.



In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create a temporary DataFrame with the necessary calculations
df_temp = pd.DataFrame()
df_temp['Date'] = pd.to_datetime(final_dates)
df_temp['click'] = data_new['click'] + data_new['basket'] + data_new['order']
df_temp['basket'] = data_new['basket'] + data_new['order']
df_temp['order'] = data_new['order']

# Group by Date and sum the values
user_activity_over_time = df_temp.groupby('Date')[['click', 'basket', 'order']].sum().reset_index()

# Create the figure
fig = go.Figure()

# Add traces for clicks, baskets, and orders
fig.add_trace(go.Scatter(x=user_activity_over_time['Date'], y=user_activity_over_time['click'],
                         mode='lines', name='Click', line=dict(width=2)))
fig.add_trace(go.Scatter(x=user_activity_over_time['Date'], y=user_activity_over_time['basket'],
                         mode='lines', name='Basket', line=dict(width=2)))
fig.add_trace(go.Scatter(x=user_activity_over_time['Date'], y=user_activity_over_time['order'],
                         mode='lines', name='Order', line=dict(width=2)))

# Update layout for better aesthetics
fig.update_layout(
    title='Analysis 10: User Activity Over Time',
    xaxis_title='Date',
    yaxis_title='User Activity',
    legend_title='Activity Type',
    width=800,
    height=500
)

# Show the plot
fig.show()


## 11. **Revenue by Manufacturer**

**Variables**: Manufacturer  
**Graph**: Bar Chart  
**Question**: Is revenue different for different manufacturers?

**Answer**:  
The manufacturers that contribute the most to revenue are 343, 175, and 239. More than a hundred manufacturers provide virtually no revenue at all.



In [ ]:
# Aggregate the data by manufacturer
manufacturer_revenue = data_new.groupby('manufacturer')['revenue'].sum().reset_index()

# Sort the data by revenue in descending order
manufacturer_revenue = manufacturer_revenue.sort_values('revenue', ascending=False)

# Take top 20 manufacturers (adjust this number as needed)
top_manufacturers = manufacturer_revenue.head(20)

# Create the bar plot with Plotly
fig = px.bar(top_manufacturers, x='manufacturer', y='revenue',
             title='Analysis 11: Revenue by Manufacturer (Top 20)',
             labels={'manufacturer': 'Manufacturer', 'revenue': 'Revenue'},
             width=1000, height=600)

# Update layout for a more polished look
fig.update_layout(
    title={'x':0.5, 'xanchor': 'center'},  # Center the title
    xaxis_title='Manufacturer',
    yaxis_title='Revenue',
    template='plotly_white',  # Cleaner background style
    xaxis={'categoryorder':'total descending'}  # Sort bars by revenue
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45)

# Add hover information
fig.update_traces(hovertemplate='Manufacturer: %{x}<br>Revenue: $%{y:,.2f}')

# Show the plot
fig.show()

## 12. **Product Genericity vs. Revenue**

**Variables**: Product Genericity and Revenue  
**Graph**: Box Plot  
**Question**: Does the general nature of a particular product affect sales?
  
**Answer**:  
The genericity of the product does seem to have a big effect on sales. The very high revenue comes from products that aren't generic. Compared to non-generic products, the median revenue of generic products is lower.

In [ ]:
fig = px.box(data_new, x='genericProduct', y='revenue',
             title=' Revenue Product vs.Genericity ',
             labels={'genericProduct': 'Product Genericity', 'revenue': 'Revenue'},
             height=500)

fig.update_layout(
    xaxis_title='Product Genericity',
    yaxis_title='Revenue',
    yaxis_tickformat=','
)

fig.show()

## 13. **Competitor Price Changes Over Time**

**Variables**: Date, Competitor's Price  
**Graph**: Line Plot  
**Question**: How has the pricing of the rival altered over time?  
**Answer**:  
Our revenue has been impacted by pricing fluctuations from the rival. For the first thirty-five days, the competitor's price has been rising; after that, its trend stays steady.



In [ ]:
import plotly.express as px

# Create the line plot
fig = px.line(data_new, x='day', y='competitorPrice',
              title='Competitor Price Changes Over Time',
              labels={'day': 'Day', 'competitorPrice': 'Competitor\'s Price'},
              height=600)

# Update layout
fig.update_layout(
    xaxis_title='Day',
    yaxis_title='Competitor Price',
    yaxis_tickformat=',.2f',  # Add comma separator for thousands and show 2 decimal places
    xaxis_tickmode='linear',  # Force all x-axis ticks to be shown
    xaxis_tick0=0,  # Start x-axis at 0
    xaxis_dtick=1  # Show every day
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show()

## 14. **User Engagement with Product**

**Variables**: Device Type, Click, Basket, Order  
**Graph**: Bar Chart  
**Question**: How do different product types affect user engagement? trend doesn't change.
**Answer**:  
The most clicked products were 10, 11, and 15, however products 10 and 11 were bought more often than product 15.


In [ ]:
import plotly.express as px

# Group data by product ID and sum clicks
df_temp = data_new.groupby('pid')['click'].sum().reset_index()

# Sort by click count and get top 20
df_temp = df_temp.sort_values('click', ascending=False).head(50)

# Create the bar plot
fig = px.bar(df_temp, x='pid', y='click',
             title='User Engagement with Top 50 Products',
             labels={'pid': 'Product ID', 'click': 'Click Count'},
             height=600)

# Update layout
fig.update_layout(
    xaxis_title='Product ID',
    yaxis_title='Click Count',
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    yaxis_tickformat=',',  # Add comma separator for thousands
    bargap=0.2  # Adjust gap between bars
)

# Show the plot
fig.show()

In [ ]:
plt.figure(figsize=(10, 6))
df_temp = data_new.groupby('pid')['order'].sum().reset_index()
df_temp = df_temp[0:20]
sns.barplot(data=df_temp, x='pid', y='order')
plt.title('Analysis 14: User Engagement with top 20 Products')
plt.xlabel('Product ID')
plt.ylabel('Order')

## 15. **Monthly Revenue Growth**

**Variables**: Date, Revenue  
**Graph**: Bar Chart  
**Question**: In what ways has revenue increased each month?  
**Sample Answer**:  
The revenue has increased steadily and modestly month over month.




In [ ]:
import plotly.express as px
import pandas as pd

# Ensure 'Date' is in datetime format
data_new['Date'] = pd.to_datetime(data_new['Date'])

# Resample data to monthly and calculate total revenue
monthly_revenue_growth = data_new.resample('M', on='Date')['revenue'].sum().reset_index()

# Create the bar plot
fig = px.bar(monthly_revenue_growth, x='Date', y='revenue',
             title='Monthly Revenue Growth',
             labels={'Date': 'Date', 'revenue': 'Revenue'},
             height=600)

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Revenue',
    yaxis_tickformat=',',
    xaxis_tickformat='%b %Y'  # Format date as 'Mon Year'
)

# Adjust bar width
fig.update_traces(marker_line_width=0, selector=dict(type="bar"))

# Show the plot
fig.show()

## 16. Click-Basket-Order Funnel Analysis

**Variables**: Click, Basket, Order  
**Graph**: Funnel Chart  
**Question**: What is the click-to-order conversion rate?  

**Sample Answer**:  
From click to order, the conversion rate is roughly 26.2%.





In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Calculate funnel steps
click_to_basket = (data_new['click'] + data_new['basket'] + data_new['order']).sum()
basket_to_order = (data_new['basket'] + data_new['order']).sum()
click_to_order = data_new['order'].sum()

# Create funnel data
funnel_data = pd.DataFrame({
    'Step': ['Click', 'Basket', 'Order'],
    'Count': [click_to_basket, basket_to_order, click_to_order]
})

# Calculate percentages
total = funnel_data['Count'].max()
funnel_data['Percentage'] = funnel_data['Count'] / total

# Create the funnel chart
fig = go.Figure(go.Funnel(
    y = funnel_data['Step'],
    x = funnel_data['Count'],
    textinfo = "value+percent initial",
    textposition = "inside",
    textfont = dict(color = "white"),
    opacity = 0.65,
    marker = {"color": ["deepskyblue", "lightsalmon", "tan"],
              "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "blue", "wheat", "wheat"]}},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}}
))

# Update layout
fig.update_layout(
    title = 'Click-Basket-Order Analysis',
    xaxis_title = 'Count',
    yaxis_title = 'Step',
    xaxis_tickformat = ',',
    height = 500
)

# Show the plot
fig.show()

## 17. **Revenue by Availability Status**

**Variables**: Availability Status, Revenue  
**Graph**: Bar Chart  
**Question**:Which user groups provide the highest income?  
**Sample Answer**:  
Availability Status 1 made about 47% of the entire revenue, which was its largest contribution.


In [ ]:
import plotly.express as px

# Group the data by availability and calculate mean revenue
availability_revenue = data_new.groupby('availability')['revenue'].mean().reset_index()

# Create the bar plot
fig = px.bar(availability_revenue, x='availability', y='revenue',
             title='Revenue According to Availability',
             labels={'availability': 'Availability Status', 'revenue': 'Average Revenue'},
             height=500)

# Update layout
fig.update_layout(
    xaxis_title='Availability Status',
    yaxis_title='Average Revenue',
    yaxis_tickformat=','
)

# Show the plot
fig.show()


## 18. **Seasonal Patterns in Revenue**

**Variables**: Date, Revenue  
**Graph**: Seasonal Decomposition Plot  
**Question**: Does income follow any seasonal trends?  
**Sample Answer**:  
There is no seasonality in revenue.

In [ ]:

# Ensure Date is in datetime format and extract month
data_new['Date'] = pd.to_datetime(data_new['Date'])
data_new['Month'] = data_new['Date'].dt.month

# Group by month and sum revenue
seasonal_revenue = data_new.groupby('Month')['revenue'].sum().reset_index()

# Create the line plot
fig = px.line(seasonal_revenue, x='Month', y='revenue',
              title='Patterns in Revenue',
              labels={'Month': 'Month', 'revenue': 'Revenue'},
              height=500)

# Update layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Revenue',
    yaxis_tickformat=',',
    xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
)

# Show the plot
fig.show()

## 19. **Impact of Advertisement on Revenue**

**Variables**: Advertisement, Revenue  
**Graph**: Box Plot  
**Question**: Does Advertisement lead to higher revenue?  
**Sample Answer**:  
Advertising generally results in a higher median revenue. However, in the revenue range of 25 to 200, the absence of advertising is associated with greater revenue.


In [ ]:
fig = px.box(data_new, x='adFlag', y='revenue',
             title='Advertisement Impact on Revenue',
             labels={'adFlag': 'Advertisement Flag', 'revenue': 'Revenue'},
             height=500)

fig.update_layout(
    xaxis_title='Ad Flag',
    yaxis_title='Revenue',
    yaxis_tickformat=','
)

fig.show()

## 20. **Content (Number of Items) vs. Order Probability**

**Variables**: Content, Basket, Order  
**Graph**: Scatter Plot  
**Question**: Does purchasing a larger number of items (bulk order) enhance the probability of placing an order?  
**Sample Answer**:  
It is not necessarily true that purchasing a greater number of items (bulk order) increases the chances of placing an order.


In [ ]:

fig = px.scatter(data_new, x='content', y='order', color='order',
                 title='Content (number of items) vs. Order Probability',
                 labels={'content': 'Content (number of items) in basket', 'order': 'Order Probability'},
                 height=500)

fig.update_layout(
    xaxis_title='Content (number of items) in basket',
    yaxis_title='Order Probability'
)

fig.show()

## 21. **Day of the Week vs. Revenue**

**Variables**: Day of the Week, Revenue  
**Graph**: Box Plot  
**Question**: Does the day of the week affect revenue?  
**Sample Answer**:  
The median revenue is generally higher on Sundays and Mondays compared to other days of the week.


In [ ]:
import plotly.express as px

# Assuming you've already created the 'Day of Week' column
data_new['Date'] = pd.to_datetime(data_new['Date'])
data_new['Day of Week'] = data_new['Date'].dt.day_name()

# Create the box plot
fig = px.box(data_new, x='Day of Week', y='revenue',
             title='Revenue vs Day of the Week ',
             labels={'Day of Week': 'Day of Week', 'revenue': 'Revenue'},
             height=500,
             category_orders={'Day of Week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']})

fig.update_layout(
    xaxis_title='Day of Week',
    yaxis_title='Revenue',
    yaxis_tickformat=','
)

fig.show()

22. **Product Popularity by Category**

Variables: Product Category, Click
Graph: Bar chart
Question: Which product categories are the most popular among users?
Sample Answer: Category C products are the most popular.


In [ ]:
features_null_values=[features for features in data_new.columns if data_new[features].isnull().sum()>1]
for i in features_null_values:
    print(i, np.round(data_new[i].isnull().mean()*100,4), '%missing values')

In [ ]:
# Print catgorical columns and count of unique values in each column
cat_cols = ['group','unit','pharmForm','salesIndex']
cat_num_cols =  ['manufacturer','adFlag','availability', "genericProduct", 'category']
for col in cat_cols+cat_num_cols:
    print( "Column: ", col , "  | Count of Unique Values: ", data_new[col].nunique() , "\n ------------" )

In [ ]:
def convert_content_column_into_numbers(item):
    try:
        return float(item)
    except:
        # extract numbers seperated by alphabet (using regex)
        import re
        item = re.findall(r'\d+\.?\d*', item)
        # multiply the numbers to get the final number
        final_number = 1
        for i in item:
            final_number = final_number*float(i)
        return final_number

data_new['content'] = data_new['content'].apply(convert_content_column_into_numbers)
data_new['content'].head()

In [ ]:
# Impute missing values in cat_num_cols categorical columns
from sklearn.impute import SimpleImputer
imputer = SimpleImputer( strategy='constant', fill_value=-1 )
data_new[cat_num_cols] = imputer.fit_transform( data_new[cat_num_cols] )

# convert the data type of cat_num_cols columns into integer
data_new[cat_num_cols] = data_new[cat_num_cols].astype(int)
data_new[cat_num_cols].head()

In [ ]:
# Impute missing values in cat_cols categorical columns
imputer = SimpleImputer( strategy='most_frequent' )
data_new[cat_cols] = imputer.fit_transform( data_new[cat_cols] )
data_new[cat_cols].head()

In [ ]:
# Convert categorical features into numbers
from sklearn.preprocessing import OrdinalEncoder
import pickle

for col in cat_cols:
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    data_new[col] = encoder.fit_transform( data_new[[col]] )
    pickle.dump( encoder, open( col+"_encoder.pkl", "wb" ) )

data_new[cat_cols].head()